In [1]:
import numpy as np 
import pandas as pd
import portion as P 
import string
import itertools
from fractions import Fraction
from scipy.optimize import linprog
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable


# Finding reductions for linear problems
Consider the following locally verifiable problem in (d, $\delta$)-biregular trees:
- $\Sigma \subseteq [0,1]$
- Task is to label the edges
- Sum of edge labels incident to
    - white nodes is $\geq \alpha$
    - black nodes is $\leq \beta$
- Additionally leaf nodes accept all possible neighborhoods.

This program tries to find a value $\{a, b, ...\}$ in each maximal separate continuous interval $A, B, C, ... \subseteq \Sigma = [0, 1]$, so that any valid labeling can be transformed to another valid labeling by **simultaneously** replacing all labels with the value corresponding to their interval. This reduces the set of labels down to $\Sigma = \{a, b, ...\}$.





The basic idea is to check every combination of these intervals and note which of these combinations could be neighborhoods of white and/or black nodes. Using these combinations we can form a system of linear inequalities that the simultanious reductions must satisfy. Lastly we use linear programming to search for a set of values.


## Example 1:

Locally verifiable problem $\Pi$ in (3,3)-biregular trees:
- $\Sigma = [0, 1/3) \cup (2/3, 1]$
- $\alpha = 1$
- $\beta = 2$

Clearly there are no 0-round solutions, as $[1/3, 2/3] \cap \Sigma = \emptyset$. Let $A= [0, 1/3)$ and $B= (2/3, 1]$. The possible combinations for a neighborhood are:

- A, A, A 
- A, A, B 
- A, B, B 
- B, B, B

We can also calculate the range of the sum of any given neighborhood:

- $A, A, A = \left[0, 1 \right)$
- $A, A, B = \left(\frac{2}{3}, \frac{5}{3}\right)$
- $A, B, B = \left(\frac{4}{3}, \frac{7}{3}\right)$
- $B, B, B = \left(2, 3\right]$

This table shows us, for example, that that the neighbourhood $A, A, B$ can be a neighbourhood of a black or a white node. This however doesn't yet mean that **any** three values from the corresponding ranges can surround both white and black nodes. It just means that if we were to collapse these ranges into single values, that collapsed neighbourhood would need to be suitable for both white and black nodes. Using this idea we can create a system of inequalities that any suitable reduction must satisfy.

\begin{align*}

a \in A \iff 0\leq a &< \frac{1}{3} \\
b \in B \iff \frac{2}{3} < b &\leq 1 \\
3a &\leq 2 \\
1 \leq 2a+b  &\leq 2 \\
1 \leq a+2b &\leq 2 \\
3b &\geq 1
\end{align*}

Now any pair $(a,b)$ satisfying these inequalities will be a valid reducion. Using linear programming, we can find for example the solution $(0.161667, 0.676667)$. Now the problem just as easy as the LCL problem (in RE syntax):
```
A A A
A A B
A B B


A A B
A B B
B B B
```

Note that these reductions **cannot** be found separately: if we choose any value for $a\in A$, we can find values $b_1, b_2, b_3 \in B$ so that a white node has the neighbourhood $a+a+b_1<1$ or a black node has the neighbourhood $a+b_2+b_3>2$, which breaks our labeling. Same logic applies to fixing $b$ first.

In [2]:
# Define some variables

# Degrees of black and white nodes
d = 3
delta = 3

# The treshold for black and white sums
beta = Fraction(10, 10)
alpha = Fraction(10, 10)

# The set of possible labels
Sigma_string = "[0, 1/3) U (2/3, 5/6) U (5/6, 1]"
#Sigma_string = "[0, 1/3) U (1/3, 21/45) U (24/45, 1]"

# Value used to handle strict inequalities in calculations. Can affect possible results. (We approximate a<b  <=> a<=b-epsilon <=> b>= a+epsilon.)
epsilon = 0.0001

# Enable/disable debug prints
debug = True

In [3]:
# Define parser for Sigma
def convert(s):
    try:
        return float(s)
    except ValueError:
        return (s)

params = {
    'disj': ' U '
}

Sigma = P.from_string(Sigma_string, conv=Fraction, **params)

In [4]:
interval_count = len([x for x in Sigma])
intervals = pd.DataFrame({"interval": [x for x in Sigma], "reduction": [None for x in Sigma]}, index=list(string.ascii_uppercase[0:interval_count]))
if debug: print(intervals)

                          interval reduction
A  [Fraction(0, 1),Fraction(1, 3))      None
B  (Fraction(2, 3),Fraction(5, 6))      None
C  (Fraction(5, 6),Fraction(1, 1)]      None


In [ ]:
class interval:
    

In [5]:
def sum_intervals(interval_list):
    min = 0
    max = 0
    min_in = True
    max_in = True
    for interval in interval_list:
        min_in = min_in and interval.left == P.CLOSED
        max_in = max_in and interval.right == P.CLOSED
        min += interval.lower
        max += interval.upper
    if min_in and max_in:
        return P.closed(min, max)
    if min_in and not max_in:
        return P.closedopen(min, max)
    if not min_in and max_in:
        return P.openclosed(min, max)
    return P.open(min, max)


In [6]:
# If white and black nodes have same amount of neighbours, some neighborhoods can be suitable for both
if d==delta:
    combinations = list(itertools.combinations_with_replacement(intervals.index, d))
    neighborhoods = pd.DataFrame({"combination": combinations})
    neighborhoods["W"] = None 
    neighborhoods["B"] = None

else:
    combinations = list(itertools.combinations_with_replacement(intervals.index, delta))
    combinations.extend(list(itertools.combinations_with_replacement(intervals.index, d)))
    neighborhoods = pd.DataFrame({"combination": combinations})
    neighborhoods["OK"] = None
    
if debug: print(neighborhoods)


  combination     W     B
0   (A, A, A)  None  None
1   (A, A, B)  None  None
2   (A, A, C)  None  None
3   (A, B, B)  None  None
4   (A, B, C)  None  None
5   (A, C, C)  None  None
6   (B, B, B)  None  None
7   (B, B, C)  None  None
8   (B, C, C)  None  None
9   (C, C, C)  None  None


In [7]:
white_range = P.closedopen(alpha, P.inf)
black_range = P.openclosed(-P.inf, beta)

if debug: 
    print(f"Black range: {black_range}")
    print(f"White range: {white_range}")
    print("\nRanges of neighbourhood sums")

for index, row in neighborhoods.iterrows():
    interval = sum_intervals(list(map(lambda x: intervals.loc[x, "interval"], row["combination"])))
    
    if debug:
        c = " ".join(row["combination"])
        print(f"{c}: {interval}" )
    # See if we have to check both white and black ranges or only one
    if d==delta:
        neighborhoods.at[index, "W"] = not (white_range & interval).empty
        neighborhoods.at[index, "B"] = not (black_range & interval).empty
        
    else: 
        if len(row["combination"]) == d:
            neighborhoods.at[index, "OK"] = not (black_range & interval).empty
            
        else: neighborhoods.at[index, "OK"] = not (white_range & interval).empty
        
if debug: print(neighborhoods)


Black range: (-inf,Fraction(1, 1)]
White range: [Fraction(1, 1),+inf)

Ranges of neighbourhood sums
A A A: [Fraction(0, 1),Fraction(1, 1))
A A B: (Fraction(2, 3),Fraction(3, 2))
A A C: (Fraction(5, 6),Fraction(5, 3))
A B B: (Fraction(4, 3),Fraction(2, 1))
A B C: (Fraction(3, 2),Fraction(13, 6))
A C C: (Fraction(5, 3),Fraction(7, 3))
B B B: (Fraction(2, 1),Fraction(5, 2))
B B C: (Fraction(13, 6),Fraction(8, 3))
B C C: (Fraction(7, 3),Fraction(17, 6))
C C C: (Fraction(5, 2),Fraction(3, 1)]
  combination      W      B
0   (A, A, A)  False   True
1   (A, A, B)   True   True
2   (A, A, C)   True   True
3   (A, B, B)   True  False
4   (A, B, C)   True  False
5   (A, C, C)   True  False
6   (B, B, B)   True  False
7   (B, B, C)   True  False
8   (B, C, C)   True  False
9   (C, C, C)   True  False


In [8]:
def detect_unions(neighborhoods):
    # Detect interchangeable letters in combinations-table.
    # If two intervals are intechangeable, a single discretization value for them can enable discretization.
    # This is a brute force solution, any smarter ideas are welcome.
    interval_count = len([x for x in Sigma])
    pairs = [string.ascii_uppercase[i:i+2] for i in range(interval_count-1)]

    eq_rows = {}

    for pair in pairs:
        eqs = []

        others = [c for c in string.ascii_uppercase[0:interval_count] if c not in pair]
        for i in range(d, 0, -1):
            eq = []
            for combination in itertools.combinations_with_replacement(pair, i):
                for end in itertools.combinations_with_replacement(others, d-i):
                    eq.append(tuple(sorted(combination+end)))
            eqs.append(eq)


        eq_rows[pair] = eqs

    interchangeables = pd.DataFrame(columns=["pair", "interchangeable"])
    interchangeables["pair"] = pairs
    interchangeables["interchangeable"] = False
    for pair in pairs:
        interchangeable = False
        for eqs in eq_rows[pair]:
            if d==delta:
                interchangeable = (neighborhoods.loc[neighborhoods["combination"].isin(eqs), ["W", "B"]].all() | ~neighborhoods.loc[neighborhoods["combination"].isin(eqs), ["W", "B"]].any()).all()
            else:
                interchangeable = (neighborhoods.loc[neighborhoods["combination"].isin(eqs), ["OK"]].all() | ~neighborhoods.loc[neighborhoods["combination"].isin(eqs), ["OK"]].any()).all()
            if not interchangeable:
                if debug: print(pair, "is not interchangeable:\n", neighborhoods.loc[neighborhoods["combination"].isin(eqs), :])
                break

        if interchangeable: interchangeables.at[interchangeables["pair"] == pair, "interchangeable"] = True


    new_intervals = "A"
    for index, row in interchangeables.iterrows():
        if not row["interchangeable"]: new_intervals += " "
        new_intervals += row["pair"][1]

    new_intervals = new_intervals.split(" ")

    # Are some intervals interchangeable (union needed)?
    return (len(new_intervals) == interval_count, new_intervals)

In [9]:
def print_retor(neighborhoods):
    if d == delta:
        white_retor = ""
        black_retor = ""
        for index, row in neighborhoods.iterrows():
            if row["W"]:
                white_retor += " ".join(row["combination"])
                white_retor += "\n"
            if row["B"]:
                black_retor += " ".join(row["combination"])
                black_retor += "\n"

    else:
        white_retor = ""
        black_retor = ""
        for index, row in neighborhoods.iterrows():
            if len(row["combination"]) == d and row["OK"]:
                black_retor += " ".join(row["combination"])
                black_retor += "\n"
            elif row["OK"]:
                white_retor += " ".join(row["combination"])
                white_retor += "\n"
    
    
    print("\nRound eliminator syntax:")
    print("\n"+black_retor)
    print("\n"+white_retor)

In [10]:
def find_reductions(neighborhoods):
    model = LpProblem(name="Reductions", sense=LpMaximize)

    variables = dict(zip(list(string.ascii_lowercase[0:interval_count]), 
                    [LpVariable(name = symbol, lowBound=0, upBound=1) for symbol in list(string.ascii_lowercase[0:interval_count])]))
    
    

    # Add the constraints of the original sets
    for index, row in intervals.iterrows():
        variable = variables[index.lower()]
        interval = row["interval"]
        if interval.left == P.CLOSED:
            model += (variable>=interval.lower, f"Init_{index}_lower")
        else:
            model += (variable>=interval.lower + epsilon, f"Init_{index}_lower")
            
        if interval.right == P.CLOSED:
            model += (variable<=interval.upper, f"Init_{index}_upper")
        else:
            model += (variable<=interval.upper - epsilon, f"Init_{index}_upper")


    # Add the constraints imposed by summation of the original sets
    if d == delta:
        for index, row in neighborhoods.iterrows():
            if (row["W"]):
                model += (sum(map(lambda x: variables[x.lower()], row["combination"])) >= alpha, f"Constraint_{index}_W")
            if (row["B"]):
                model += (sum(map(lambda x: variables[x.lower()], row["combination"])) <= beta, f"Constraint_{index}_B")

    else:  
        for index, row in neighborhoods.iterrows():
            if (len(row["combination"])==d) and row["OK"]:
                model += (sum(map(lambda x: variables[x.lower()], row["combination"])) <= beta, f"Constraint_{index}_B")
            elif (row["OK"]):
                model += (sum(map(lambda x: variables[x.lower()], row["combination"])) >= alpha, f"Constraint_{index}_W")


    if debug: print("\n", model, "\n")

    # Search for solutions to the given system of inequalities
    if model.solve() == -1:
        print("No reductions found.")
    
    else:
        print("Reductions found: ")
        for var in model.variables()[1:]:
            intervals.loc[var.name.upper(), "reduction"] = var.value()
        
        print(intervals)
        print_retor(neighborhoods)
        


In [11]:
def run_reductor():
    # First check for 0-round solution
    easy_solution_interval = P.closed(Fraction(alpha, delta), Fraction(beta, d))
    easy_solutions = (easy_solution_interval & Sigma)
    if not easy_solutions.empty:
        print("0-round solution found.")
        print(f"Choose any single value from {P.to_string(easy_solutions, **params)}.")
    
    # Try to find some reductions
    else:
        print("No 0-round solutions found.")
        find_reductions(neighborhoods)
    print(detect_unions(neighborhoods))

    


In [12]:
run_reductor()

No 0-round solutions found.

 Reductions:
MAXIMIZE
None
SUBJECT TO
Init_A_lower: a >= 0

Init_A_upper: a <= 0.333233333333

Init_B_lower: b >= 0.666766666667

Init_B_upper: b <= 0.833233333333

Init_C_lower: c >= 0.833433333333

Init_C_upper: c <= 1

Constraint_0_B: 3 a <= 1

Constraint_1_W: 2 a + b >= 1

Constraint_1_B: 2 a + b <= 1

Constraint_2_W: 2 a + c >= 1

Constraint_2_B: 2 a + c <= 1

Constraint_3_W: a + 2 b >= 1

Constraint_4_W: a + b + c >= 1

Constraint_5_W: a + 2 c >= 1

Constraint_6_W: 3 b >= 1

Constraint_7_W: 2 b + c >= 1

Constraint_8_W: b + 2 c >= 1

Constraint_9_W: 3 c >= 1

VARIABLES
a <= 1 Continuous
b <= 1 Continuous
c <= 1 Continuous
 

No reductions found.
AB is not interchangeable:
   combination      W      B
0   (A, A, A)  False   True
1   (A, A, B)   True   True
3   (A, B, B)   True  False
6   (B, B, B)   True  False
(False, ['A', 'BC'])
